# Curso Computer Vision

<img src="https://yaelmanuel.com/wp-content/uploads/2021/12/platzi-banner-logo-matematicas.png" width="500px">

---

**Instalar dependencias**

In [2]:
!pip install mediapipe

**Definir path del video**

In [3]:
video_path = "/content/head-pose-face-detection-male.mp4"

In [4]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt

def add_gaussian_to_heatmap(heatmap, center, sigma, amplitude):
    """
    Agrega un parche gaussiano al heatmap en la posición 'center'.
    heatmap: matriz de activación (h x w).
    center: (cx, cy) centro del parche.
    sigma: desviación estándar de la gaussiana.
    amplitude: valor máximo a sumar.
    """
    h, w = heatmap.shape
    y, x = np.indices((h, w))
    cx, cy = center
    gaussian = amplitude * np.exp(-((x - cx)**2 + (y - cy)**2) / (2 * sigma**2))
    heatmap += gaussian
    return heatmap

# Parámetros
output_video_path = "output_pose_estimation.mp4"  # Archivo de salida
decay_factor = 0.98      # Factor de decaimiento para el heatmap (entre 0 y 1)
sigma = 15               # Desviación estándar para el parche gaussiano
amplitude = 50           # Amplitud del parche gaussiano

# Inicialización de Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5,
                                  min_tracking_confidence=0.5,
                                  )

# Abrir video de entrada y preparar VideoWriter
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
h, w, _ = frame.shape

# Inicializar heatmap de seguimiento de miradas
heatmap_gaze = np.zeros((h, w), dtype=np.float32)

# Guardar el archivo output con formato mp4
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Aplicar decay para atenuar activaciones antiguas
    heatmap_gaze *= decay_factor

    # Procesar frame para estimar pose (Face Mesh)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    resultados = face_mesh.process(frame_rgb)

    if resultados.multi_face_landmarks:
        for face_landmarks in resultados.multi_face_landmarks:
            # Índices típicos para los ojos: 33 (ojo izquierdo) y 263 (ojo derecho)
            left_eye = face_landmarks.landmark[33]
            right_eye = face_landmarks.landmark[263]
            left_eye_coords = (int(left_eye.x * w), int(left_eye.y * h))
            right_eye_coords = (int(right_eye.x * w), int(right_eye.y * h))

            # Dibujar los ojos en el frame (opcional)
            cv2.circle(frame, left_eye_coords, 3, (0, 255, 0), -1)
            cv2.circle(frame, right_eye_coords, 3, (0, 255, 0), -1)

            # Aproximar el punto medio entre los ojos (proxy para la dirección de mirada)
            mid_eye = (int((left_eye_coords[0] + right_eye_coords[0]) / 2),
                       int((left_eye_coords[1] + right_eye_coords[1]) / 2))

            # Sumar un parche gaussiano al heatmap en la posición del mid_eye
            heatmap_gaze = add_gaussian_to_heatmap(heatmap_gaze, mid_eye, sigma, amplitude)

    # Generar una visualización del heatmap
    heatmap_vis = cv2.normalize(heatmap_gaze, None, 0, 255, cv2.NORM_MINMAX)
    heatmap_vis = np.uint8(heatmap_vis)
    colored_heatmap = cv2.applyColorMap(heatmap_vis, cv2.COLORMAP_JET)

    # Superponer el heatmap al frame original
    overlay = cv2.addWeighted(frame, 0.6, colored_heatmap, 0.4, 0)

    # Escribir el frame procesado en el video de salida
    writer.write(overlay)

cap.release()
writer.release()
face_mesh.close()

print(f"Video procesado y guardado en: {output_video_path}")

Video procesado y guardado en: output_pose_estimation.mp4
